# Steps

SMART BRFSS Data Processing
- columns preprocessing
- select topic with on 'depression'
- deal with nan values for answer (1 - no%) >  yes, if applicable
- check and merge with the metropolitandivisioncode/cbsacode, maximize the match numbers


[Behavioral Risk Factors: Selected Metropolitan Area Risk Trends (SMART) MMSA Prevalence Data (2011 to Present)](https://data.cdc.gov/Behavioral-Risk-Factors/Behavioral-Risk-Factors-Selected-Metropolitan-Area/j32a-sa6u/about_data)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
pd.set_option('display.max_columns', None)

In [ ]:
data_dir = "../00_data/01_raw/"
interm_dir = "../00_data/02_intermediate/"

In [3]:
smart = pd.read_csv(f"{data_dir}Behavioral_Risk_Factors__Selected_Metropolitan_Area_Risk_Trends__SMART__MMSA_Prevalence_Data__2011_to_Present__20241004.csv")

In [4]:
smart.Topic.unique()

array(['Disability status', 'Hearing', 'Physical Activity Index',
       'HIV Test', 'Tetanus Shot', 'Cardiovascular Disease',
       'Overall Health', 'Health Care Coverage', 'Seatbelt Use',
       'Cholesterol Checked', 'Healthy Days', 'COPD', 'Asthma',
       'Veteran Status', 'PSA Test', 'Last Checkup', 'Diabetes',
       'Skin Cancer', 'Other Cancer', 'BMI Categories', 'Smoker Status',
       'Teeth Removed', 'All Teeth Removed', 'Flu Shot', 'Exercise',
       'Vegetable Consumption', 'Alcohol Consumption',
       'Fair or Poor Health', 'Kidney', 'Health Care Cost',
       'Current Smoker Status', 'Pneumonia Vaccination', 'Arthritis',
       'Blood Stool Test', 'Mammogram', 'Smokeless Tobacco', 'Depression',
       'High Blood Pressure', 'Vision', 'Binge Drinking',
       'Drink and Drive', 'USPSTF Recommendations', 'Heavy Drinking',
       'Personal Care Provider', 'Dental Visit', 'Under 65 Coverage',
       'Pap Test', 'Aerobic Activity', 'Cholesterol High',
       'Fruit Consum

## Data Preprocessing

In [5]:
# filter out depression topic only
smart_depr = smart[smart.Topic == 'Depression']
smart_depr.head(2)

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,RESPONSEID,GeoLocation
200,2011,15804,"Camden, NJ Metropolitan Division",Chronic Health Indicators,Depression,Ever told you that you have a form of depression?,Yes,Overall,Overall,253,12.52,10.34,14.70,50,%,Crude Prevalence,NaN,NaN,BRFSS,CLASS03,Topic17,15804,BO1,CAT1,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)"
240,2020,41700,"San Antonio-New Braunfels, TX Metropolitan Sta...",Chronic Health Indicators,Depression,Ever told you that you have a form of depression?,No,Overall,Overall,516,81.42,77.17,85.67,10,%,Crude Prevalence,NaN,NaN,BRFSS,CLASS03,TOPIC17,41700,BO1,CAT1,ADDEPEV3,RESP054,"(29.4330549, -98.6069656)"


In [6]:
smart_depr.shape

(3432, 27)

In [7]:
smart_depr.groupby('Year')['Locationdesc'].nunique()


Year
2011    198
2012    187
2013    145
2014    132
2015    130
2016    143
2017    136
2018    134
2019    136
2020    118
2021    126
2022    131
Name: Locationdesc, dtype: int64

In [8]:
smart_depr.groupby('Year')['Locationdesc'].nunique()

Year
2011    198
2012    187
2013    145
2014    132
2015    130
2016    143
2017    136
2018    134
2019    136
2020    118
2021    126
2022    131
Name: Locationdesc, dtype: int64

In [9]:
len(smart_depr.Locationdesc.unique())

270

In [10]:
smart_depr.groupby('Year').size()

Year
2011    396
2012    374
2013    290
2014    264
2015    260
2016    286
2017    272
2018    268
2019    272
2020    236
2021    252
2022    262
dtype: int64

In [11]:
# check unique values of each column
# for col, values in {col: smart_depr[col].unique() for col in smart_depr.columns}.items():
#     print(f"Unique values in column '{col}': {values}")

In [12]:
def process_columns(df, selected_columns_to_keep):
    """
    This function counts unique values for each column in the dataframe (ignoring case for strings),
    memorizes the columns with only 1 unique value (case insensitive), includes columns with all NaN values,
    excludes the selected columns, and prints the remaining columns with their original unique value(s).
    
    Parameters:
    - df: DataFrame to process
    - selected_columns_to_keep: list of column names to exclude from the final results
    
    Returns:
    - A list of columns that have only 1 unique value (case insensitive) or are all NaN,
      and were not selected to keep.
    """
    
    columns_with_one_unique_value = []

    for col in df.columns:
        if df[col].dtype == 'object':
            # Convert strings to lowercase and get unique values
            unique_values_lower = df[col].str.lower().unique()
            unique_values_original = df[col].unique()
        else:
            unique_values_lower = unique_values_original = df[col].unique()

        # Check if column has only one unique value or all NaN values
        if len(unique_values_lower) == 1 or df[col].isna().all():
            columns_with_one_unique_value.append((col, unique_values_original))

    # Exclude selected columns to keep
    columns_with_one_unique_value = [
        (col, values) for col, values in columns_with_one_unique_value if col not in selected_columns_to_keep
    ]

    # Print columns with their unique values
    for col, unique_values in columns_with_one_unique_value:
        print(f"Column '{col}' has only 1 unique value(s) or is all NaN: {list(unique_values)}")

    # Return the column names
    return [col for col, _ in columns_with_one_unique_value]

In [13]:
columns_to_drop = process_columns(smart_depr, ['Data_value_unit', 'Data_value_type', 'Data_Value_Footnote'])
smart_depr_selected = smart_depr.drop(columns=columns_to_drop)
smart_depr_selected.head(3)

Column 'Class' has only 1 unique value(s) or is all NaN: ['Chronic Health Indicators']
Column 'Topic' has only 1 unique value(s) or is all NaN: ['Depression']
Column 'Question' has only 1 unique value(s) or is all NaN: ['Ever told you that you have a form of depression?']
Column 'Break_Out' has only 1 unique value(s) or is all NaN: ['Overall']
Column 'Break_Out_Category' has only 1 unique value(s) or is all NaN: ['Overall']
Column 'DataSource' has only 1 unique value(s) or is all NaN: ['BRFSS']
Column 'ClassId' has only 1 unique value(s) or is all NaN: ['CLASS03']
Column 'TopicId' has only 1 unique value(s) or is all NaN: ['Topic17', 'TOPIC17']
Column 'BreakoutID' has only 1 unique value(s) or is all NaN: ['BO1']
Column 'BreakOutCategoryID' has only 1 unique value(s) or is all NaN: ['CAT1']


,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation
200,2011,15804,"Camden, NJ Metropolitan Division",Yes,253,12.52,10.34,14.70,50,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)"
240,2020,41700,"San Antonio-New Braunfels, TX Metropolitan Sta...",No,516,81.42,77.17,85.67,10,%,Crude Prevalence,NaN,NaN,41700,ADDEPEV3,RESP054,"(29.4330549, -98.6069656)"
256,2011,41420,"Salem, OR Metropolitan Statistical Area",No,448,75.99,71.46,80.52,50,%,Crude Prevalence,NaN,NaN,41420,ADDEPEV2,RESP054,"(44.9033791, -122.9017427)"


In [14]:
smart_depr.Data_Value_Footnote_Symbol.unique()

array([nan, '*'], dtype=object)

In [15]:
smart_depr.Data_Value_Footnote.unique()

array([nan,
       'Prevalence estimate not available if the unweighted sample size for the denominator was < 50 or the Relative Standard Error (RSE) is > 0.3 or if the state did not collect data for that calendar year.'],
      dtype=object)

In [16]:
smart_depr.loc[smart_depr.Data_Value_Footnote_Symbol == '*']

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,RESPONSEID,GeoLocation
30063,2020,18580,"Corpus Christi, TX Metropolitan Statistical Area",Chronic Health Indicators,Depression,Ever told you that you have a form of depression?,Yes,Overall,Overall,156,NaN,NaN,NaN,2,%,Crude Prevalence,*,Prevalence estimate not available if the unwei...,BRFSS,CLASS03,TOPIC17,18580,BO1,CAT1,ADDEPEV3,RESP046,"(27.76688, -97.40815)"
134086,2016,17780,"College Station-Bryan, TX Metropolitan Statist...",Chronic Health Indicators,Depression,Ever told you that you have a form of depression?,Yes,Overall,Overall,59,NaN,NaN,NaN,1,%,Crude Prevalence,*,Prevalence estimate not available if the unwei...,BRFSS,CLASS03,TOPIC17,17780,BO1,CAT1,ADDEPEV2,RESP046,"(30.62816, -96.33712)"


### Only Select Yes Answers for Depression Topic

In [17]:
# Iterate through the rows where Response is 'Yes' and Data_value is NaN
for index, row in smart_depr_selected[smart_depr_selected['Response'] == 'Yes'].iterrows():
    location = row['Locationdesc']
    year = row['Year']
    
    # Find the corresponding 'No' response for the same location and year
    no_value = smart_depr_selected[(smart_depr_selected['Locationdesc'] == location) & 
                                   (smart_depr_selected['Year'] == year) & 
                                   (smart_depr_selected['Response'] == 'No')]['Data_value'].values
    
    # If a 'No' response exists and Yes is NaN, replace Yes with 100 - No
    if len(no_value) > 0 and pd.isna(row['Data_value']):
        smart_depr_selected.at[index, 'Data_value'] = 100 - no_value[0]

smart_depr_selected_yes = smart_depr_selected[smart_depr_selected.Response == 'Yes']
print('smart_depr shape:', smart_depr.shape)
print('smart_depr_selected shape:', smart_depr_selected.shape)
print('smart_depr_selected_yes shape:', smart_depr_selected_yes.shape)
print('smart_depr_selected_yes max year:', smart_depr_selected_yes .Year.max(), 'smart_depr_selected_yes min year:', smart_depr_selected_yes .Year.min())

smart_depr shape: (3432, 27)
smart_depr_selected shape: (3432, 17)
smart_depr_selected_yes shape: (1716, 17)
smart_depr_selected_yes max year: 2022 smart_depr_selected_yes min year: 2011


In [18]:
smart_depr_selected_yes.nlargest(5, 'Data_value')

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation
188495,2022,28700,"Kingsport-Bristol-Bristol, TN-VA Metropolitan ...",Yes,183,36.85,31.19,42.51,2,%,Crude Prevalence,NaN,NaN,28700,ADDEPEV3,RESP046,"(36.6041558, -82.4401118)"
184666,2022,16860,"Chattanooga, TN-GA Metropolitan Statistical Area",Yes,176,33.28,28.30,38.26,2,%,Crude Prevalence,NaN,NaN,16860,ADDEPEV3,RESP046,"(35.0489417, -85.3611582)"
188615,2022,28940,"Knoxville, TN Metropolitan Statistical Area",Yes,188,32.23,27.60,36.86,2,%,Crude Prevalence,NaN,NaN,28940,ADDEPEV3,RESP046,"(35.9294445, -84.0154928)"
14017,2019,13740,"Billings, MT Metropolitan Statistical Area",Yes,238,31.04,27.41,34.67,2,%,Crude Prevalence,NaN,NaN,13740,ADDEPEV3,RESP046,"(45.6371402, -108.8323611)"
4719,2019,28700,"Kingsport-Bristol-Bristol, TN-VA Metropolitan ...",Yes,168,30.62,25.82,35.42,2,%,Crude Prevalence,NaN,NaN,28700,ADDEPEV3,RESP046,"(36.6041611, -82.4401446)"


In [19]:
smart_depr_selected_yes.nsmallest(5, 'Data_value')

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation
79101,2013,41940,"San Jose-Sunnyvale-Santa Clara, CA Metropolita...",Yes,66,7.77,5.52,10.02,49,%,Crude Prevalence,NaN,NaN,41940,ADDEPEV2,RESP046,"(36.9084669, -121.3713591)"
139943,2016,35004,"Nassau County-Suffolk County, NY Metropolitan ...",Yes,101,8.60,6.31,10.89,1,%,Crude Prevalence,NaN,NaN,35004,ADDEPEV2,RESP046,"(40.9057199, -72.8348299)"
58436,2011,42044,"Santa Ana-Anaheim-Irvine, CA Metropolitan Divi...",Yes,121,9.09,6.82,11.36,50,%,Crude Prevalence,NaN,NaN,42044,ADDEPEV2,RESP046,"(33.6756824, -117.7771947)"
76858,2012,11244,"Los Angeles-Long Beach-Anaheim, CA Metropolita...",Yes,113,9.20,6.93,11.47,41,%,Crude Prevalence,NaN,NaN,11244,ADDEPEV2,RESP046,"(34.065193, -118.244222)"
56755,2011,26180,"Honolulu, HI Metropolitan Statistical Area",Yes,389,9.58,8.21,10.95,50,%,Crude Prevalence,NaN,NaN,26180,ADDEPEV2,RESP046,"(27.606, -105.718)"


In [20]:
smart_depr_selected_yes[smart_depr_selected_yes[['Year','Locationabbr']].duplicated(keep=False)] # no duplicate

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation


In [21]:
# Check if Locationabbr and LocationID columns are identical
are_columns_identical = smart_depr_selected_yes['Locationabbr'].equals(smart_depr_selected_yes['LocationID'])

print("Are Locationabbr and LocationID columns identical?", are_columns_identical)

Are Locationabbr and LocationID columns identical? True


## Merge with fips

SMART merge with cbsa code: [cbsa-csa-fips-county-crosswalk](https://data.nber.org/cbsa-csa-fips-county-crosswalk/cbsa2fipsxw.csv)

In [22]:
smart_depr_selected_yes['Locationdesc']

200                        Camden, NJ Metropolitan Division
283               Rutland, VT Micropolitan Statistical Area
328          Fort Worth-Arlington, TX Metropolitan Division
619              Aberdeen, SD Micropolitan Statistical Area
689          Cedar Rapids, IA Metropolitan Statistical Area
                                ...                        
196875          Wenatchee, WA Metropolitan Statistical Area
196993            Wichita, KS Metropolitan Statistical Area
197113           Wilmington, DE-MD-NJ Metropolitan Division
197233       Worcester, MA-CT Metropolitan Statistical Area
197353    Youngstown-Warren-Boardman, OH-PA Metropolitan...
Name: Locationdesc, Length: 1716, dtype: object

In [23]:
keywords = ["Metropolitan Division", "Micropolitan Statistical Area", "Metropolitan Statistical Area"]

# Create a function to split based on these keywords
def split_location(value):
    for keyword in keywords:
        if keyword in value:
            # Split by the keyword and strip any extra spaces
            city_state, area_type = value.rsplit(keyword, 1)[0].strip(), keyword
            return city_state, area_type
    return value, np.nan  # If no keyword is found, return the original value with NaN

# Apply the function and create new columns
smart_depr_selected_yes = smart_depr_selected_yes.copy()
smart_depr_selected_yes[['location_name', 'type']] = smart_depr_selected_yes['Locationdesc'].apply(lambda x: pd.Series(split_location(x)))


In [24]:
smart_depr_selected_yes.type.unique()

array(['Metropolitan Division', 'Micropolitan Statistical Area',
       'Metropolitan Statistical Area'], dtype=object)

In [25]:
smart_depr_selected_yes.type.isna().sum()

np.int64(0)

In [26]:
cbsa = pd.read_csv(f"{data_dir}cbsa2fipsxw.csv")
cbsa_raw = cbsa.copy()
cbsa.head(5)

,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,33860,NaN,388.0,"Montgomery, AL",Metropolitan Statistical Area,NaN,"Montgomery-Selma, AL",Autauga County,Alabama,1,1,Central
1,19300,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1,3,Central
2,21640,NaN,NaN,"Eufaula, AL-GA",Micropolitan Statistical Area,NaN,NaN,Barbour County,Alabama,1,5,Central
3,13820,NaN,142.0,"Birmingham, AL",Metropolitan Statistical Area,NaN,"Birmingham-Cullman-Talladega, AL",Bibb County,Alabama,1,7,Outlying
4,13820,NaN,142.0,"Birmingham, AL",Metropolitan Statistical Area,NaN,"Birmingham-Cullman-Talladega, AL",Blount County,Alabama,1,9,Outlying


### Deal with NaN in metropolitandivisioncode

In [27]:
smart_depr_selected_yes.Locationdesc.nunique(), smart_depr_selected_yes.Locationabbr.nunique(), cbsa.shape

(270, 268, (1915, 12))

In [28]:
cbsa['metropolitandivisioncode'].isna().sum()

np.int64(1776)

In [29]:
cbsa['metropolitandivisioncode'] = cbsa['metropolitandivisioncode'].fillna(cbsa['cbsacode']) # brutal force
cbsa.head(3) 

,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,33860,33860.0,388.0,"Montgomery, AL",Metropolitan Statistical Area,NaN,"Montgomery-Selma, AL",Autauga County,Alabama,1,1,Central
1,19300,19300.0,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1,3,Central
2,21640,21640.0,NaN,"Eufaula, AL-GA",Micropolitan Statistical Area,NaN,NaN,Barbour County,Alabama,1,5,Central


In [30]:
cbsa = cbsa[["cbsacode", "metropolitandivisioncode", "cbsatitle"]]

In [31]:
smart_depr_selected_yes.shape, cbsa.shape

((1716, 19), (1915, 3))

In [32]:
smart_depr_selected_yes.Locationabbr.dtypes

dtype('int64')

In [33]:
inner_merge = pd.merge(
    smart_depr_selected_yes, cbsa,
    left_on='Locationabbr', right_on='metropolitandivisioncode',
    how='inner'
)

unmatched_rows = smart_depr_selected_yes[
    ~smart_depr_selected_yes['Locationabbr'].isin(inner_merge['Locationabbr'])
]

print("Unmatched rows after primary inner merge:")
print(len(unmatched_rows))

if not unmatched_rows.empty:
    alternative_merge = pd.merge(
        unmatched_rows, cbsa,
        left_on='Locationabbr', right_on='cbsacode', # alternative column # brutal merge
        how='inner'
    )

    final_result = pd.concat([inner_merge, alternative_merge]).drop_duplicates()
else:
    final_result = inner_merge


still_unmatched = smart_depr_selected_yes[
    ~smart_depr_selected_yes['Locationabbr'].isin(final_result['Locationabbr'])
]

print("Still-unmatched rows after both merges:")
print(len(still_unmatched))


Unmatched rows after primary inner merge:
131
Still-unmatched rows after both merges:
74


In [34]:
# Separate `smart_depr_selected_yes` based on `type`
msa_or_msa = smart_depr_selected_yes[smart_depr_selected_yes['type'].isin(['Metropolitan Statistical Area', 'Micropolitan Statistical Area'])]
metropolitan_division = smart_depr_selected_yes[smart_depr_selected_yes['type'] == 'Metropolitan Division']

# Merge based on `Locationabbr` and `cbsacode` for Metropolitan Statistical Area and Micropolitan Statistical Area
merge_msa_or_msa = msa_or_msa.merge(cbsa_raw, left_on='Locationabbr', right_on='cbsacode', how='left')

# Merge based on `Locationabbr` and `metropolitandivisioncode` for Metropolitan Division
merge_metropolitan_division = metropolitan_division.merge(cbsa_raw, left_on='Locationabbr', right_on='metropolitandivisioncode', how='left')

# Concatenate the results
merged_result = pd.concat([merge_msa_or_msa, merge_metropolitan_division], ignore_index=True)

In [35]:
merged_result.cbsacode.isna().sum()

np.int64(74)

In [36]:
missing = merged_result[merged_result['cbsacode'].isna()].copy()

In [37]:
final_result.head()

,Year,Locationabbr,Locationdesc,Response,Sample_Size,Data_value,Confidence_limit_Low,Confidence_limit_High,Display_order,Data_value_unit,Data_value_type,Data_Value_Footnote_Symbol,Data_Value_Footnote,LocationID,QuestionID,RESPONSEID,GeoLocation,location_name,type,cbsacode,metropolitandivisioncode,cbsatitle
0,2011,15804,"Camden, NJ Metropolitan Division",Yes,253,12.52,10.34,14.70,50,%,Crude Prevalence,NaN,NaN,15804,ADDEPEV2,RESP046,"(39.8098807, -74.8232086)","Camden, NJ",Metropolitan Division,37980,15804.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD"
3,2011,40860,"Rutland, VT Micropolitan Statistical Area",Yes,153,21.61,17.44,25.78,49,%,Crude Prevalence,NaN,NaN,40860,ADDEPEV2,RESP046,"(43.5808351, -73.0381951)","Rutland, VT",Micropolitan Statistical Area,40860,40860.0,"Rutland, VT"
4,2020,23104,"Fort Worth-Arlington, TX Metropolitan Division",Yes,120,18.22,14.61,21.83,2,%,Crude Prevalence,NaN,NaN,23104,ADDEPEV3,RESP046,"(32.8073735, -97.537101)","Fort Worth-Arlington, TX",Metropolitan Division,19100,23104.0,"Dallas-Fort Worth-Arlington, TX"
8,2020,10100,"Aberdeen, SD Micropolitan Statistical Area",Yes,92,18.73,13.93,23.53,2,%,Crude Prevalence,NaN,NaN,10100,ADDEPEV3,RESP046,"(45.5194353, -98.7007003)","Aberdeen, SD",Micropolitan Statistical Area,10100,10100.0,"Aberdeen, SD"
10,2020,16300,"Cedar Rapids, IA Metropolitan Statistical Area",Yes,162,19.61,16.57,22.65,2,%,Crude Prevalence,NaN,NaN,16300,ADDEPEV3,RESP046,"(42.0855843, -91.6304636)","Cedar Rapids, IA",Metropolitan Statistical Area,16300,16300.0,"Cedar Rapids, IA"


# Merge by geolocation

In [38]:
cbsa_shape = gpd.read_file(f"{data_dir}cbsa_shape/tl_2024_us_cbsa.shp")

In [39]:
cbsa_shape

,CSAFP,CBSAFP,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,MEMI,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,490,11640,11640,310M700US11640,"Arecibo, PR","Arecibo, PR Metro Area",M1,1,G3110,613528952,231467259,+18.4392239,-066.7599813,"POLYGON ((-66.96346 18.47302, -66.96337 18.473..."
1,None,11660,11660,310M700US11660,"Arkadelphia, AR","Arkadelphia, AR Micro Area",M2,2,G3110,2242855174,43965580,+34.0533162,-093.1762085,"POLYGON ((-93.50221 34.21413, -93.50221 34.214..."
2,556,11680,11680,310M700US11680,"Arkansas City-Winfield, KS","Arkansas City-Winfield, KS Micro Area",M2,2,G3110,2915648172,17322934,+37.2345068,-096.8372468,"POLYGON ((-96.82596 37.47613, -96.81016 37.476..."
3,120,11700,11700,310M700US11700,"Asheville, NC","Asheville, NC Metro Area",M1,1,G3110,3830729933,19336007,+35.6163405,-082.5706835,"POLYGON ((-82.4086 35.81818, -82.40815 35.8183..."
4,360,11740,11740,310M700US11740,"Ashland, OH","Ashland, OH Micro Area",M2,2,G3110,1095528191,9969109,+40.8432730,-082.2701271,"POLYGON ((-82.22067 40.66736, -82.22112 40.667..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,218,37800,37800,310M700US37800,"Pella, IA","Pella, IA Micro Area",M2,2,G3110,1436123089,41484609,+41.3314327,-093.0938678,"POLYGON ((-93.09922 41.16087, -93.10304 41.160..."
931,None,37860,37860,310M700US37860,"Pensacola-Ferry Pass-Brent, FL","Pensacola-Ferry Pass-Brent, FL Metro Area",M1,1,G3110,4323723291,1757254546,+30.6085566,-087.1585466,"POLYGON ((-86.91833 30.21764, -86.92528 30.216..."
932,427,37900,37900,310M700US37900,"Peoria, IL","Peoria, IL Metro Area",M1,1,G3110,6391181524,130518754,+40.7880685,-089.5146389,"POLYGON ((-89.04724 41.0133, -89.04724 41.0128..."
933,294,37940,37940,310M700US37940,"Peru, IN","Peru, IN Micro Area",M2,2,G3110,968246955,9184623,+40.7728833,-086.0442575,"POLYGON ((-86.01332 40.56413, -86.01355 40.564..."


In [40]:
missing[['latitude', 'longitude']] = missing['GeoLocation'].str.extract(r'\(([^,]+), ([^)]+)\)').astype(float)
missing['geometry'] = missing.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
missing_gdf = gpd.GeoDataFrame(missing, geometry='geometry')

In [41]:
missing_gdf = missing_gdf.set_crs("EPSG:4326")  # WGS 84
cbsa_shape = cbsa_shape.to_crs("EPSG:4326")

In [42]:
smerged_result = gpd.sjoin(missing_gdf, cbsa_shape, how="left", predicate="within")

In [43]:
smerged_result.GEOID.isna().sum()

np.int64(6)

In [44]:
# Fill missing 'cbsacode' values with 'GEOID'
smerged_result['cbsacode'] = smerged_result['cbsacode'].fillna(smerged_result['GEOID'])

# Fill missing 'cbsatitle' values with 'NAME'
smerged_result['cbsa_title'] = smerged_result['cbsatitle'].fillna(smerged_result['NAME'])

smerged_result.drop([
    'latitude', 'longitude', 'geometry', 'index_right', 'CSAFP', 'CBSAFP', 
    'GEOID', 'GEOIDFQ', 'NAME', 'NAMELSAD', 'LSAD', 'MEMI', 'MTFCC', 
    'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'
], axis=1, inplace=True)

In [45]:
find = merged_result[merged_result['cbsacode'].notna()].copy()
final_result_1 = pd.concat([smerged_result, find], ignore_index=True)

# Agg into CBSA level

In [46]:
# Define a function to calculate the weighted mean
def weighted_mean(x):
    return np.average(x, weights=final_result_1.loc[x.index, "Sample_Size"])

# Group by the specified columns and aggregate
grouped_result = final_result_1.groupby(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit']).agg(
    Data_value=('Data_value', weighted_mean),
    Confidence_limit_Low=('Confidence_limit_Low', weighted_mean),
    Confidence_limit_High=('Confidence_limit_High', weighted_mean),
    Sample_Size=('Sample_Size', 'sum')
).reset_index()

In [47]:
grouped_result

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size
0,10100.0,2011,"Aberdeen, SD",%,16.75,11.83,21.67,176
1,10100.0,2014,"Aberdeen, SD",%,19.36,14.17,24.55,210
2,10100.0,2015,"Aberdeen, SD",%,14.29,10.57,18.01,168
3,10100.0,2017,"Aberdeen, SD",%,16.05,10.66,21.44,146
4,10100.0,2018,"Aberdeen, SD",%,20.73,15.95,25.51,178
...,...,...,...,...,...,...,...,...
1540,49660.0,2018,"Youngstown-Warren, OH",%,21.67,17.12,26.22,236
1541,49660.0,2019,"Youngstown-Warren, OH",%,21.98,17.55,26.41,236
1542,49660.0,2020,"Youngstown-Warren, OH",%,22.97,18.95,26.99,284
1543,49660.0,2021,"Youngstown-Warren, OH",%,23.74,19.17,28.31,284


In [48]:
cbsa_years = grouped_result.groupby("cbsacode")["Year"].nunique().reset_index()
cbsa_across_all_years = cbsa_years[cbsa_years["Year"] == 12]["cbsacode"]
print("CBSA codes present across all 12 years:",len(cbsa_across_all_years))

CBSA codes present across all 12 years: 73


In [56]:
allYears = cbsa_across_all_years.tolist()
filtered_grouped_result = grouped_result[grouped_result['cbsacode'].isin(allYears)].copy()
filtered_grouped_result.to_csv(f"{interm_dir}smart_with_cbsa_all_12_years.csv", index=False)

In [57]:
filtered_grouped_result

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size
31,10740.0,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880
32,10740.0,2012,"Albuquerque, NM",%,21.22,19.48,22.96,2876
33,10740.0,2013,"Albuquerque, NM",%,20.25,18.05,22.45,1748
34,10740.0,2014,"Albuquerque, NM",%,22.79,20.22,25.36,1664
35,10740.0,2015,"Albuquerque, NM",%,20.84,18.10,23.58,1300
...,...,...,...,...,...,...,...,...
1531,49340.0,2018,"Worcester, MA",%,18.85,15.79,21.91,270
1532,49340.0,2019,"Worcester, MA",%,20.10,17.20,23.00,263
1533,49340.0,2020,"Worcester, MA",%,18.31,14.98,21.64,257
1534,49340.0,2021,"Worcester, MA",%,20.31,17.17,23.45,302


# Additional url might be helpful:

https://www2.census.gov/programs-surveys/cbp/technical-documentation/reference/metro-area-geography-reference/msa_county_reference22.txt